In [1]:
###Reference keywords from preprocessing of CLIMBERT pretrained model###
reference_tokens_climate_BERT=['’emissions’, ”’, ’temperature’, ’environmental’,’soil’, ’increase’, ’conditions’, ’potential’, ’increased’, ’areas’, ’degrees’, ’across’, ’systems’, ’emission’, ’precipitation’, ’impacts’, ’compared’, ’countries’, ’sustainable’,’provide’, ’reduction’, ’annual’, ’reduce’, ’greenhouse’,’approach’, ’processes’, ’factors’, ’observed’, ’renewable’,’temperatures’, ’distribution’, ’studies’, ’variability’, ’significantly’, ’–’, ’further’, ’regions’, ’addition’, ’showed’,’“’, ’industry’, ’consumption’, ’regional’, ’risks’, ’atmospheric’, ’supply’, ’companies’, ’plants’, ’biomass’, ’electricity’, ’respectively’, ’activities’, ’communities’, ’climatic’, ’solar’, ’investment’, ’spatial’, ’rainfall’, ’•’, ’sustainability’, ’costs’, ’reduced’, ’2021’, ’influence’, ’vegetation’, ’sources’, ’possible’, ’ecosystem’, ’scenarios’, ’summer’, ’drought’, ’structure’, ’economy’, ’considered’, ’various’, ’atmosphere’, ’several’, ’technologies’, ’transition’,’assessment’, ’dioxide’, ’ocean’, ’fossil’, ’patterns’, ’waste’,’solutions’, ’transport’, ’strategy’, ’CH4’, ’policies’, ’understanding’, ’concentration’, ’customers’, ’methane’, ’applied’, ’increases’, ’estimated’, ’flood’, ’measured’, ’thermal’, ’concentrations’, ’decrease’, ’greater’, ’following’,’proposed’, ’trends’, ’basis’, ’provides’, ’operations’, ’differences’, ’hydrogen’, ’adaptation’, ’methods’, ’capture’,’variation’, ’reducing’, ’N2O’, ’parameters’, ’ecosystems’,’investigated’, ’yield’, ’strategies’, ’indicate’, ’caused’, ’dynamics’, ’obtained’, ’efforts’, ’coastal’, ’become’, ’agricultural’, ’decreased’, ’GHG’, ’materials’, ’mainly’, ’relationship’, ’ecological’, ’benefits’, ’+/-’, ’challenges’, ’nitrogen’, ’forests’, ’trend’, ’estimates’, ’towards’, ’Committee’,’seasonal’, ’developing’, ’particular’, ’importance’, ’tropical’, ’ratio’, ’2030’, ’composition’, ’employees’, ’characteristics’, ’scenario’, ’measurements’, ’plans’, ’fuels’, ’infrastructure’, ’overall’, ’responses’, ’presented’, ’least’, ’assess’, ’diversity’, ’periods’, ’delta’, ’included’, ’already’,’targets’, ’achieve’, ’affect’, ’conducted’, ’operating’, ’populations’, ’variations’, ’studied’, ’additional’, ’construction’,’northern’, ’variables’, ’soils’, ’ensure’, ’recovery’, ’combined’, ’decision’, ’practices’, ’however’, ’determined’, ’resulting’, ’mitigation’, ’conservation’, ’estimate’, ’identify’,’observations’, ’losses’, ’productivity’, ’agreement’, ’monitoring’, ’investments’, ’pollution’, ’contribution’, ’opportunities’, ’simulations’, ’gases’, ’statements’, ’planning’,’shares’, ’sediment’, ’flux’, ’requirements’, ’trees’, ’temporal’, ’determine’, ’southern’, ’previous’, ’integrated’, ’relatively’, ’analyses’, ’means’, ’2050’, ’”’, ’uncertainty’,’pandemic’, ’fluxes’, ’findings’, ’moisture’, ’consistent’,’decades’, ’snow’, ’performed’, ’contribute’, ’crisis’']
token_data=reference_tokens_climate_BERT[0]
list_words=token_data.split(',')
new_keywords=[]
for key in list_words:
    if len(key)>3:
        key=key.replace('’','').strip()
        new_keywords.append(key)
            

In [2]:
len(new_keywords)

232

In [3]:
import pandas as pd
import transformers
import PyPDF2
import numpy as np
import re
import os


num_matches = 3

def get_words(input):
    return re.compile('\w+').findall(input)

###Lets train a embedding model on the dataset of 20 pdfs (sample) to get the vector representation of the words/sentences

training_data_path=''
final_train_data=[]                   
##Finding relevent pages of information 
for filename in os.listdir(training_data_path):  
    if filename.endswith('.pdf'):
        # Open the PDF file
        with open(os.path.join(training_data_path, filename), 'rb') as pdf_file:
            try:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                # Iterate over each page
                for page_num in range(len(pdf_reader.pages)):
                    if True:
                        page = pdf_reader.pages[page_num]
                        text = page.extract_text()
                        pattern1 = r'\bScope ?[1-3]'
                        pattern2=r'\b(?:tCo2e|tCO2e|teqCO2|metric tons|CO2 emissions|tonnes CO2e)\b'
                        total_useful_sentence=[]
                        #if len(matches2)>0 or len(matches1)>0:
                        sentences = text.split('.')
                        sentences_cleaned = [' '.join(text.split()) for text in sentences]
                        if len(sentences_cleaned)>0:
                            for sent in sentences_cleaned:
                                if len(sent.split())> 25:
                                    matches1 = re.findall(pattern1,sent,re.IGNORECASE)
                                    matches2 = re.findall(pattern2,sent,re.IGNORECASE)
                                    keyword_count=0
                                #for words in new_keywords[:100]:
                                #    if words in sent:
                                #        keyword_count=keyword_count+1
                                    if len(matches1)>0 or len(matches2)>0: #or keyword_count>=5:
                                    
                                    #print('number of useful sentences on page number ',page_num,'are ',len(total_useful_sentence))
                                        final_train_data.append([filename,page_num,sent,len(matches1),len(matches2),keyword_count])
            except:
                print('loading error',filename)
                #except:
                #    print('Error loading data')

In [4]:
df=pd.DataFrame(final_train_data,columns=['company','page_num','content','Scope match','Metric match','keywor_count'])

In [6]:
df.to_csv('training_dataset.csv')

In [7]:
df

,company,page_num,content,Scope match,Metric match,keywor_count
0,AC-FR_2019_universal-registration-document.pdf,173,"Scopes 1, 2 and 3 are included in this study w...",1,0,0
1,AC-FR_2019_universal-registration-document.pdf,174,●●scope 2 covers the indirect emissions from t...,2,0,0
2,AC-FR_2019_universal-registration-document.pdf,174,Results 2017 carbon footprint Direct emissions...,6,5,0
3,AC-FR_2019_universal-registration-document.pdf,174,"28 teqCO2 per room and per year These results,...",0,1,0
4,AC-FR_2019_universal-registration-document.pdf,174,Breakdown of emissions by scope – Accor Group ...,3,0,0
...,...,...,...,...,...,...
179,DLVEY-US_2020_annual-report.pdf,165,"GRI 302: Energy 103-1, 103-2, 103-3 Management...",3,0,0
180,DNBBY-US_2019_annual-report.pdf,30,We follow the GHG Protocol Corporate Standard ...,3,0,0
181,DNBBY-US_2019_annual-report.pdf,30,8 RECs (renewable certificates) or GOs (guaran...,1,0,0
182,DNBBY-US_2019_annual-report.pdf,55,Scope 1 includes direct emissions from sources...,2,0,0


In [8]:
# Lets try to add embeddings and reduce its dimensions to 2 for visualization

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
  
stop_words = set(stopwords.words('english'))
from gensim.utils import simple_preprocess
content=df['content'].values.tolist()
tokenized =[]
for sentence in content:
  # the simple_preprocess function returns a list of each sentence
  tokenized.append(simple_preprocess(sentence, deacc = True))

filtered_toknzd=[]
for tk in tokenized:
    filtered_sentence = [w for w in tk if not w.lower() in stop_words]
    filtered_toknzd.append(filtered_sentence)

In [9]:
filtered_toknzd

[['scopes',
  'included',
  'study',
  'following',
  'scopes',
  'scope',
  'covers',
  'emissions',
  'buildings',
  'owned',
  'operated',
  'group',
  'vehicles',
  'owned',
  'group',
  'fuel',
  'consumption',
  'universal',
  'registration',
  'document',
  'accorcorporate',
  'responsibility',
  'aiming',
  'carbon',
  'neutrality',
  'hotel',
  'network',
  'ccor',
  'brand'],
 ['scope',
  'covers',
  'indirect',
  'emissions',
  'production',
  'energy',
  'purchased',
  'operations',
  'electricity',
  'purchased',
  'steam',
  'heating',
  'air',
  'conditioning',
  'scope',
  'covers',
  'indirect',
  'emissions',
  'due',
  'upstream',
  'downstream',
  'activities',
  'required',
  'group',
  'direct',
  'activity',
  'upstream',
  'activities',
  'purchase',
  'goods',
  'services',
  'capital',
  'goods',
  'indirect',
  'fuel',
  'energy',
  'consumption',
  'waste',
  'generated',
  'activities',
  'business',
  'travel',
  'downstream',
  'activities',
  'franchises

In [10]:
import gensim.downloader as api
from multiprocessing import cpu_count
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
 
#print('The length of data is =====>',data)
# We will split the data into two parts
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(filtered_toknzd)]
model = Doc2Vec(tagged_data, vector_size = 40, window = 4, min_count = 1, epochs = 100)

test_doc = word_tokenize("emissions are very high".lower())
test_doc_vector = model.infer_vector(test_doc)

In [72]:
model.save('Doc2Vec.model')

In [11]:
doc_token=[]

for sent in df['content'].values.tolist():
    test_doc = word_tokenize(sent.lower())
    doc_token.append(model.infer_vector(test_doc))

In [12]:
df['new_token']=doc_token

In [13]:
df_tkn=pd.DataFrame(doc_token)

In [14]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
pca = PCA(n_components=2)
pca.fit(df_tkn)
print(pca.explained_variance_ratio_)
tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
vis_dims = tsne.fit_transform(doc_token)
vis_dims.shape

[0.1382301  0.08931196]


(184, 2)

In [15]:
df_new=pd.DataFrame()
df_new['text']=df['content']
df_new[['x','y']]=vis_dims

In [16]:
df_new.to_csv('')

In [17]:
df_new[['Scope match','Metric match']]=df[['Scope match','Metric match']]

In [145]:
df_123=pd.read_csv()
df_12=pd.read_csv()

In [146]:
df_123['label']='Scope_1_2_3_compliant'
df_12['label']='Scope_1_2_compliant'

In [144]:
df_123_label=df_123[['text','label']]
df_12_label=df_12[['text','label']]

In [83]:
json_train_1=df_123_label.to_dict('records')
json_train_2=df_12_label.to_dict('records')

In [84]:
json_train_1.extend(json_train_2)

In [92]:
import json
with open(r'','w') as file:
    json.dump(json_train_1,file)

In [147]:
x=pd.concat([df_123,df_12],axis=0)

In [148]:
x

,Unnamed: 0,text,x,y,label
0,0,"Scopes 1, 2 and 3 are included in this study w...",-5.361702,14.247186,Scope_1_2_3_compliant
1,1,●●scope 2 covers the indirect emissions from t...,-7.160213,18.953194,Scope_1_2_3_compliant
2,3,"28 teqCO2 per room and per year These results,...",-4.428095,11.816884,Scope_1_2_3_compliant
3,4,Breakdown of emissions by scope – Accor Group ...,1.125077,15.985170,Scope_1_2_3_compliant
4,5,Breakdown of greenhouse gas emissions by hotel...,-1.317030,13.060192,Scope_1_2_3_compliant
...,...,...,...,...,...
79,176,"As pre- scribed by the GHG Protocol, we consid...",7.181840,-5.256165,Scope_1_2_compliant
80,178,GREENHOUSE GAS EMISSIONS 3 Scope 1 EmissionsSc...,-1.318450,-4.101784,Scope_1_2_compliant
81,180,We follow the GHG Protocol Corporate Standard ...,-2.503074,-6.732966,Scope_1_2_compliant
82,181,8 RECs (renewable certificates) or GOs (guaran...,11.765115,-5.766499,Scope_1_2_compliant


In [150]:
x.replace({'Scope_1_2_3_compliant':1,'Scope_1_2_compliant':0},inplace=True)

In [161]:
#use of logistic regression for classication of scop1_2 and scope_1_2_3
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

X, y = x[['x','y']],x[['label']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.predict(X_test)

pickle.dump(clf, open(r'', 'wb'))

C:\python\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# USE OF PRETRAINED MODELS

# using other pre-trained library

In [ ]:
#importing dataset

In [1]:
import json
import pandas as pd
f = open(r'')
data = json.load(f)
data=pd.DataFrame(data)

In [2]:
data

,text,label
0,"Scopes 1, 2 and 3 are included in this study w...",Scope_1_2_3_compliant
1,●●scope 2 covers the indirect emissions from t...,Scope_1_2_3_compliant
2,"28 teqCO2 per room and per year These results,...",Scope_1_2_3_compliant
3,Breakdown of emissions by scope – Accor Group ...,Scope_1_2_3_compliant
4,Breakdown of greenhouse gas emissions by hotel...,Scope_1_2_3_compliant
...,...,...
171,"As pre- scribed by the GHG Protocol, we consid...",Scope_1_2_compliant
172,GREENHOUSE GAS EMISSIONS 3 Scope 1 EmissionsSc...,Scope_1_2_compliant
173,We follow the GHG Protocol Corporate Standard ...,Scope_1_2_compliant
174,8 RECs (renewable certificates) or GOs (guaran...,Scope_1_2_compliant


In [3]:
data.replace({'Scope_1_2_3_compliant':1,'Scope_1_2_compliant':0},inplace=True)

In [4]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [5]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 176
})

In [6]:
def process_data(row):
    text = row['text']
    text = str(text)
    text = ' '.join(text.split())
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = 0
    if row['label'] == 1:
        label += 1
    encodings['label'] = label
    encodings['text'] = text
    return encodings

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('climatebert/distilroberta-base-climate-f')

In [8]:
tokenizer

RobertaTokenizerFast(name_or_path='climatebert/distilroberta-base-climate-f', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [9]:
print(process_data({
    'text': 'emission based on carbon footprints are measured',
    'label': 'positive'
}))

{'input_ids': [0, 50279, 716, 15, 4363, 35372, 32, 1437, 50364, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': 0, 'text': 'emission based on carbon footprints are measured'}


In [10]:
processed_data = []
for i in range(len(data)):
    processed_data.append(process_data(data.iloc[i]))

In [11]:
processed_data

[{'input_ids': [0, 12645, 18836, 112, 6, 132, 8, 155, 32, 1437, 50434, 11, 42, 892, 19, 5, 1437, 50369, 2850, 18836, 35, 30022, 49093, 39576, 112, 4865, 1437, 50266, 31, 3413, 2164, 50, 7187, 30, 5, 826, 8, 1734, 2164, 30, 5, 826, 36, 20118, 1437, 50307, 4397, 28628, 93, 954, 9973, 43812, 50416, 282, 27246, 111, 10018, 3411, 15228, 34044, 2640, 4374, 179, 534, 13, 512, 35994, 18755, 13, 5, 2303, 1546, 223, 5, 10, 740, 7215, 7379, 176, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': 

In [12]:
from sklearn.model_selection import train_test_split
import pyarrow as pa
from datasets import Dataset

new_df = pd.DataFrame(processed_data)
train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=20
)
train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [13]:
from transformers import AutoModelForSequenceClassification #TFAutoModelForSequenceClassification==> For tensorflow

model = AutoModelForSequenceClassification.from_pretrained(
    'climatebert/distilroberta-base-climate-f',
    num_labels=2
)

Some weights of the model checkpoint at climatebert/distilroberta-base-climate-f were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifie

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\python\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 140
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 54
  Number of trainable parameters = 82300418
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` metho

Epoch,Training Loss,Validation Loss
1,No log,0.704243
2,No log,0.655275
3,No log,0.548656


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 36
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 36
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forwa

TrainOutput(global_step=54, training_loss=0.6346075976336444, metrics={'train_runtime': 241.4383, 'train_samples_per_second': 1.74, 'train_steps_per_second': 0.224, 'total_flos': 13909076858880.0, 'train_loss': 0.6346075976336444, 'epoch': 3.0})

In [38]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 36
  Batch size = 8


{'eval_loss': 0.5831831097602844,
 'eval_runtime': 6.8314,
 'eval_samples_per_second': 5.27,
 'eval_steps_per_second': 0.732,
 'epoch': 3.0}

In [39]:
model.save_pretrained(r'C:\Users\Rideema Malji\OneDrive\Desktop\Others\Upwork\data labeling\sample_training_testmodel\pretrained_BERT_model')

Configuration saved in C:\Users\Rideema Malji\OneDrive\Desktop\Others\Upwork\data labeling\sample_training_testmodel\pretrained_BERT_model\config.json
Model weights saved in C:\Users\Rideema Malji\OneDrive\Desktop\Others\Upwork\data labeling\sample_training_testmodel\pretrained_BERT_model\pytorch_model.bin


In [40]:
##load pretrained CLIMATEBERT model : https://huggingface.co/climatebert/distilroberta-base-climate-f/tree/main

# Get predictions

In [41]:
import torch
import numpy as np

def get_prediction(text):
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = model(**encoding)

    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)
    
    if label == 1:
        return {
            'Label': 'Scop1_2_compliant',
            'probability': probs[1]
        }
    else:
        return {
            'Label': 'Scop1_2_3_compliant',
            'probability': probs[0]
        }

In [42]:
get_prediction('The Co2 emissions for new ways of industries are rising and they contributed mostly from Electricity and power distribution')

{'Label': 'Scop1_2_compliant', 'probability': 0.51219946}